In [ ]:
import numpy as np
import os
import cv2
import json
import glob
from PIL import Image

import youtube_dl
from __future__ import unicode_literals

## Download video from youtube

In [1]:
ydl_opts = {}
with youtube_dl.YoutubeDL(ydl_opts) as ydl:
    ydl.download(['https://www.youtube.com/watch?v=1ovAjgh2ezM'])

[youtube] 1ovAjgh2ezM: Downloading webpage
[youtube] 1ovAjgh2ezM: Downloading player 3804dce2
[download] Destination: 100 лет за три минуты — история танца-1ovAjgh2ezM.mp4
[download] 100% of 10.95MiB in 00:0015MiB/s ETA 00:001


In [15]:
vidcap = cv2.VideoCapture('100 лет за три минуты — история танца-1ovAjgh2ezM.mp4')
fps    = vidcap.get(cv2.CAP_PROP_FPS)

In [16]:
length_of_video = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
width  = int(vidcap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(vidcap.get(cv2.CAP_PROP_FRAME_HEIGHT))
print('fps:', fps)
print('length_of_video:', length_of_video)
print('width:', width)
print('height:', height)

fps: 25.0
length_of_video: 5003
width: 640
height: 360


In [ ]:
success,image = vidcap.read()
count = 0
while success:
    cv2.imwrite("frames/frame_%d.jpg" % count, image)     # save frame as JPEG file      
    success,image = vidcap.read()
    #print('Read a new frame: ', success)
    count += 1
    #break

## Pose estimation

In [23]:
frames_list = ['frames/' + item for item in os.listdir('frames/') if 'ipynb_checkpoints' not in item]
frames_list = ' '.join(frames_list)

In [109]:
# test image
img_pth = 'frames/frame_2115.jpg' 

In [50]:
# make predictions
! python lightweight-human-pose-estimation.pytorch/demo.py --checkpoint-path weights/checkpoint_iter_370000.pth --images $frames_list

In [75]:
# write json for future processing 
data = {}
counter = 0
img_list = []

pose_frames = os.listdir('./processed/')
for i in range(5003):
    if 'frame_{}_0.jpg'.format(i) in pose_frames:
        counter += 1
        img_new = './processed/frame_{}_0.jpg'.format(str(i))
        img_list = [img_new] + img_list[:9]
    if 'frame_{}_1.jpg'.format(i) in pose_frames:
        counter += 1
        img_new = './processed/frame_{}_1.jpg'.format(str(i))
        img_list = [img_new] + img_list[:9]
    data[i] = {'hands up': counter,
              'last 10': img_list}
    
with open('pose_estimation.json', 'w') as f:
    json.dump(data, f)

## Create concatenated frames

In [52]:
data = {}
counter = 0
img_list = ['./concatenated_frames/background.jpg'] * 10

pose_frames = os.listdir('./processed/')
for i in range(5003):
    if 'frame_{}_0.jpg'.format(i) in pose_frames:
        counter += 1
        img_new = './processed/frame_{}_0.jpg'.format(str(i))
        img_list = [img_new] + img_list[:9]
    if 'frame_{}_1.jpg'.format(i) in pose_frames:
        counter += 1
        img_new = './processed/frame_{}_1.jpg'.format(str(i))
        img_list = [img_new] + img_list[:9]
    data[i] = {'hands up': counter,
              'last 10': img_list}

In [86]:
def get_concat(img_list):
    dst = Image.new('RGB', (250, 200))
    for h in range(2):
        for w in range(5):
            dst.paste(img_list[h*5 + w], (50*w, 100*h))
    return dst

for i in range(5003):
    img_list = []
    for img_pth in data[i]['last 10']:
        im1 = Image.open(img_pth)
        size = int((im1.width / max(im1.width, im1.height))* 100.), int((im1.height / max(im1.width, im1.height))* 100.)
        img_list += [im1.resize(size, Image.ANTIALIAS)]
    
    get_concat(img_list).save('concatenated_frames/frame_{}.jpg'.format(i))

## Run server

In [98]:
! python app.py

 * Serving Flask app 'app' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
 * Running on all addresses.
 * Running on http://192.168.1.77:5000/ (Press CTRL+C to quit)
 * Restarting with stat
 * Debugger is active!
 * Debugger PIN: 126-150-024
192.168.1.77 - - [26/Jul/2021 19:42:46] "GET / HTTP/1.1" 200 -
192.168.1.77 - - [26/Jul/2021 19:42:46] "POST /get_data HTTP/1.1" 200 -
192.168.1.77 - - [26/Jul/2021 19:42:47] "GET /image_feed HTTP/1.1" 200 -
192.168.1.77 - - [26/Jul/2021 19:42:48] "GET /video_feed HTTP/1.1" 200 -
192.168.1.77 - - [26/Jul/2021 19:42:49] "POST /get_data HTTP/1.1" 200 -
192.168.1.77 - - [26/Jul/2021 19:42:52] "POST /get_data HTTP/1.1" 200 -
192.168.1.77 - - [26/Jul/2021 19:42:55] "POST /get_data HTTP/1.1" 200 -
192.168.1.77 - - [26/Jul/2021 19:42:58] "POST /get_data HTTP/1.1" 200 -
192.168.1.77 - - [26/Jul/2021 19:43:01] "POST /get_data HTTP/1.1" 200 -
192.168.1.77 - - [26/Jul/2021 19:43:04] "POST /get_data HTTP/1.

## Save mp4

In [107]:
img_array = []
for i in range(5003):
    filename = 'frames_with_pose/frame_{}.jpg'.format(i)
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width,height)
    img_array.append(img)
 
 
out = cv2.VideoWriter('project.mp4',cv2.VideoWriter_fourcc(*'DIVX'), 15, size)
 
for i in range(len(img_array)):
    out.write(img_array[i])
out.release()

## Save gif

In [103]:
def make_gif(frame_folder):
    filename = []
    for i in range(5003):
        filename += ['frames_with_pose/frame_{}.jpg'.format(i)]
    frames = [Image.open(image) for image in filename[:1000]]
    frame_one = frames[0]
    frame_one.save("flask_demo.gif", format="GIF", append_images=frames,
                   save_all=True, duration=50, loop=0)
    
if __name__ == "__main__":
    make_gif("output")